In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#import json

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim

#from pandas.io.json import json_normalize

#import matplotlib.cm as cm
#import matplotlib.colors as colors

#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
#import folium 

print('Libraries imported.')


Libraries imported.


In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', context=ctx).read()

In [4]:
soup = BeautifulSoup(html, "html.parser")

### Get table from soup and convert to dataframe

In [10]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove boroughs with status 'Not assigned'

In [11]:
filter = df["Borough"] != "Not assigned"
df = df[filter]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Change 'Not assigned' neighbourhood to applicable borough

In [12]:
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Group postcodes together and combine neighbourhoods

In [13]:
df = df.groupby(['Postcode','Borough']).agg(lambda Neighbourhood: ', '.join(Neighbourhood))
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df.shape

(103, 3)